In [1]:
from sqlalchemy import create_engine, text
import pandas as pd

In [2]:
username = 'sa'
password = '1'
hostname = 'DESKTOP-6BJG2TB'
port = '1433'
database_destination = 'DWH'

In [7]:
json = {'before': {'id': 1, 'first_name': 'Julee', 'last_name': 'Lequeux', 'email': 'jlequeux0@latimes.com', 'gender': 'Male', 'ip_address': '56.235.32.50'}, 'after': {'id': 1, 'first_name': 'Julee', 'last_name': 'Lequeux', 'email': 'jlequeux0@latimes.com', 'gender': 'Female', 'ip_address': '56.235.32.50'}, 'source': {'version': '2.0.1.Final', 'connector': 'sqlserver', 'name': 'DBCustomers', 'ts_ms': 1681095620260, 'snapshot': 'false', 'db': 'Customers', 'sequence': None, 'schema': 'dbo', 'table': 'CustomerDetails', 'change_lsn': '00000035:00006448:0002', 'commit_lsn': '00000035:00006448:0003', 'event_serial_no': 2}, 'op': 'u', 'ts_ms': 1681095623399, 'transaction': None}

In [8]:
df = pd.DataFrame([json])
df

,before,after,source,op,ts_ms,transaction
0,"{'id': 1, 'first_name': 'Julee', 'last_name': ...","{'id': 1, 'first_name': 'Julee', 'last_name': ...","{'version': '2.0.1.Final', 'connector': 'sqlse...",u,1681095623399,None


In [11]:
cdc_table = pd.DataFrame([df.loc[0,'before']])
cdc_table

,id,first_name,last_name,email,gender,ip_address
0,1,Julee,Lequeux,jlequeux0@latimes.com,Male,56.235.32.50


In [10]:
engine = create_engine(f'mssql+pymssql://{username}:{password}@{hostname}:{port}/{database_destination}')
cdc_table.to_sql('temp', engine, if_exists='replace')

sql = "UPDATE CustomerDetails" + \
" SET first_name = t.first_name," + \
" last_name = t.last_name," + \
" email = t.email," + \
" gender = t.gender," + \
" ip_address = t.ip_address" + \
" FROM temp t WHERE CustomerDetails.id = t.id"

with engine.begin() as conn:
    conn.execute(text(sql))